In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
class EVEIApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Data Visualization App")
        self.root.geometry("500x500")

        self.tabControl = ttk.Notebook(root)

        self.tab1 = ttk.Frame(self.tabControl)
        self.tabControl.add(self.tab1, text='File Selection')
        
        self.tabMC = ttk.Frame(self.tabControl)
        self.tabControl.add(self.tabMC, text='MC')

        self.tabPhysicsBox = ttk.Frame(self.tabControl)
        self.tabControl.add(self.tabPhysicsBox, text='PhysicsBox')

        self.tabControl.pack(expand=1, fill="both")

        self.create_file_selection_tab()

    def create_file_selection_tab(self):
        self.load_MC_button = tk.Button(self.tab1, text="Load MC Excel File", font=('Helvetica', 18), command=lambda: self.load_file("MC"))
        self.load_MC_button.pack(pady=30)

        self.load_PhysicsBox_button = tk.Button(self.tab1, text="Load PhysicsBox Excel File", font=('Helvetica', 18), command=lambda: self.load_file("PhysicsBox"))
        self.load_PhysicsBox_button.pack(pady=30)

    def load_file(self, file_type):
        file_path = filedialog.askopenfilename()

        if file_type == "MC":
            data = pd.read_excel(file_path, skiprows=4, engine='openpyxl')
            self.plot_hobart_data(data)
        elif file_type == "PhysicsBox":
            data = pd.read_excel(file_path, skiprows=2, engine='openpyxl')
            self.plot_physicsbox_data(data)

    def plot_hobart_data(self, data):

            data['Battery Power'] = data['Battery Voltage'] * data['Battery Current']

            # Calculate the Motor Power consumption: P = V * I
            data['Motor Power'] = data['Motor Voltage'] * data['Motor Current']

            for widget in self.tabMC.winfo_children():
                widget.destroy()

            back_to_selection_button = tk.Button(self.tabMC, text="Back to File Selection", command=self.go_back)
            back_to_selection_button.pack(side=tk.TOP, pady=10)

            to_physics_button = tk.Button(self.tabMC, text="Go to PhysicsBox", command=self.go_to_physicsbox)
            to_physics_button.pack(side=tk.TOP, pady=10)

            self.tabControl.select(self.tabMC)    
            # Plot the graph in 2x2 grid
            fig, axs = plt.subplots(2, 2, figsize=(10,10))

            # Motor Power Consumption
            axs[0, 0].plot(data['t'], data['Motor Power'], label='Motor Power Consumption')
            axs[0, 0].set_title('Motor Power Consumption vs t')
            axs[0, 0].set_xlabel('Time (t)')
            axs[0, 0].set_ylabel('Power Consumption')
            axs[0, 0].grid(True)
            axs[0, 0].legend()

            # Motor Temperature
            axs[0, 1].plot(data['t'], data['M- Temperature'], label='M- Temperature', color='r')
            axs[0, 1].set_title('M- Temperature vs t')
            axs[0, 1].set_xlabel('Time (t)')
            axs[0, 1].set_ylabel('Temperature')
            axs[0, 1].grid(True)
            axs[0, 1].legend()

            # Battery Power Consumption
            axs[1, 0].plot(data['t'], data['Battery Power'], label='Battery Power Consumption')
            axs[1, 0].set_title('Battery Power Consumption vs t')
            axs[1, 0].set_xlabel('Time (t)')
            axs[1, 0].set_ylabel('Power Consumption')
            axs[1, 0].grid(True)
            axs[1, 0].legend()

            # Battery Temperature
            axs[1, 1].plot(data['t'], data['B+ Temperature'], label='B+ Temperature', color='g')
            axs[1, 1].set_title('B+ Temperature vs t')
            axs[1, 1].set_xlabel('Time (t)')
            axs[1, 1].set_ylabel('Temperature')
            axs[1, 1].grid(True)
            axs[1, 1].legend()

            fig.tight_layout(pad=4.0)  # Add padding between plots for better visualization

            # Clear previous figures and widgets on the canvas

            canvas = FigureCanvasTkAgg(fig, master=self.tabMC)
            canvas_widget = canvas.get_tk_widget()
            canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=True)


    def plot_physicsbox_data(self, data):
        fig, axs = plt.subplots(2, 1, figsize=(10,10))
        
        
        back_to_selection_button = tk.Button(self.tabPhysicsBox, text="Back to File Selection", command=self.go_back)
        back_to_selection_button.pack(side=tk.TOP, pady=10)

        to_MC_button = tk.Button(self.tabPhysicsBox, text="Go to MC", command=self.go_to_MC)
        to_MC_button.pack(side=tk.TOP, pady=10)
        
        
        self.tabControl.select(self.tabPhysicsBox)
        # Power over time
        axs[0].plot(data['millis']/1000, data['Power(kW)'], label='Power over time')
        axs[0].set_title('Power over time')
        axs[0].set_xlabel('Time (s)')
        axs[0].set_ylabel('Power(kW)')
        axs[0].grid(True)
        axs[0].legend()

        # RPM over time
        axs[1].plot(data['millis'], data['RPM'], label='RPM over time', color='r')
        axs[1].set_title('RPM over time')
        axs[1].set_xlabel('Time (millis)')
        axs[1].set_ylabel('RPM')
        axs[1].grid(True)
        axs[1].legend()

        fig.tight_layout(pad=4.0)

        canvas = FigureCanvasTkAgg(fig, master=self.tabPhysicsBox)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    def go_back(self):
        self.tabControl.select(self.tab1)

    def go_to_physicsbox(self):
        self.tabControl.select(self.tabPhysicsBox)

    def go_to_MC(self):
        self.tabControl.select(self.tabMC)

if __name__ == "__main__":
    root = tk.Tk()
    app = EVEIApp(root)
    root.mainloop()
